In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
from pathlib import Path
import cv2

In [5]:
# Paths
zip_file_path = r"C:\Users\shree\Downloads\archive (13).zip"
extracted_path = r"/content/braille_dataset"

# Unzip the dataset
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

In [6]:
# Load images
image_dir = Path('/content/braille_dataset/Braille Dataset/Braille Dataset')
dir_list = list(image_dir.glob('*.jpg'))
image_count = len(dir_list)
print(f"Total images: {image_count}")

Total images: 1560


In [7]:
# Read images and store in a list
images = []
name_list = []
for file_path in dir_list:
    I = cv2.imread(str(file_path))
    images.append(I)
    # Extracting label from the filename
    name_list.append(os.path.basename(file_path)[0])

In [8]:
# Convert images and labels to numpy arrays
images_list = np.array(images)
name_list = np.array(name_list).T

# Normalize images
images_list = images_list / 255.0
print(images_list[0])

[[[0.76078431 0.76078431 0.76078431]
  [0.8        0.8        0.8       ]
  [0.80784314 0.80784314 0.80784314]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 [[0.75686275 0.75686275 0.75686275]
  [0.69411765 0.69411765 0.69411765]
  [0.7372549  0.7372549  0.7372549 ]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 [[0.78039216 0.78039216 0.78039216]
  [0.79607843 0.79607843 0.79607843]
  [0.76470588 0.76470588 0.76470588]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 ...

 [[0.76862745 0.76862745 0.76862745]
  [0.77254902 0.77254902 0.77254902]
  [0.76470588 0.76470588 0.76470588]
  ...
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]
  [0.76862745 0.76862745 0.76862745]]

 [[0.77254902 0.77254902 0.77254902]
  [0.77647059 0.77647059 0.77647059]


In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode labels (character labels into numeric values)
le = LabelEncoder()
name_list = le.fit_transform(name_list)

# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(images_list, name_list, test_size=0.2, random_state=42)

# Defining class names for characters a to z
class_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [10]:
import tensorflow as tf
from tensorflow import keras

# Define the CNN model with 3 Conv2D layers followed by MaxPooling, BatchNormalization, and Dense layers
model = keras.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(28,28,3)),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Flatten(),
    keras.layers.Dense(units=512, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=26, activation="softmax")
])

C:\Users\shree\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model with Adam optimizer, Sparse Categorical Crossentropy loss, and accuracy metric
model.compile(optimizer="Adam", loss="SparseCategoricalCrossentropy", metrics=["sparse_categorical_accuracy"])

# Train the model with 20 epochs, using 30% of training data for validation
history = model.fit(x=X_train, y=y_train, epochs=20, validation_split=0.3)

Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step - loss: 3.8214 - sparse_categorical_accuracy: 0.2198 - val_loss: 3.4312 - val_sparse_categorical_accuracy: 0.0427
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - loss: 1.4683 - sparse_categorical_accuracy: 0.5981 - val_loss: 4.9099 - val_sparse_categorical_accuracy: 0.0427
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - loss: 0.8186 - sparse_categorical_accuracy: 0.7581 - val_loss: 6.8225 - val_sparse_categorical_accuracy: 0.0427
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - loss: 0.5876 - sparse_categorical_accuracy: 0.8275 - val_loss: 10.5641 - val_sparse_categorical_accuracy: 0.0427
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 95ms/step - loss: 0.4056 - sparse_categorical_accuracy: 0.8710 - val_loss: 10.2743 - val_sparse_categorical_accuracy: 0.0427
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step - loss: 0.3035 - sparse_categorical_accuracy: 0.9085 - val_loss: 12.0764 - val_sparse_categorical_accuracy: 0.0427
Epoch 7

In [ ]:
# Print model summary
model.summary()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Plot training and validation loss over epochs
time = np.arange(1, len(history.history['loss'])+1)

# Loss plot
sns.lineplot(data=history.history, x=time, y='loss')
sns.lineplot(data=history.history, x=time, y='val_loss')
plt.title('Loss fitting history')
plt.legend(labels=['Loss', 'Validation loss'])
plt.show()

# Accuracy plot
sns.lineplot(data=history.history, x=time, y='sparse_categorical_accuracy')
sns.lineplot(data=history.history, x=time, y='val_sparse_categorical_accuracy')
plt.title('Accuracy fitting history')
plt.legend(labels=['Accuracy', 'Validation accuracy'])
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
from random import randrange

# Select a random image from the test set
index_image = randrange(len(X_test))

# Make a prediction
prediction_scores = model.predict(np.expand_dims(X_test[index_image], axis=0))
predicted_index = np.argmax(prediction_scores)

# Display the image and predicted label
plt.imshow(X_test[index_image])
plt.title(f"Predicted label: {class_names[predicted_index]}")
plt.show()

# Print prediction confidence
print("Prediction scores:", prediction_scores)
print("Predicted label:", class_names[predicted_index])